## Function to modify the distance between plates

In [2]:
import re
import os

def modify_plates_distance(new_gap):
    with open("geometry.geo", "r") as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        # Match Rectangle A (upper)
        if re.match(r'\s*Rectangle\(1\)', line):
            # Calculate new Y = gap / 2
            y = new_gap / 2
            new_line = f'Rectangle(1) = {{-50, {y}+2, 0, 100, 4, 0}};\n'
            new_lines.append(new_line)
        # Match Rectangle B (lower)
        elif re.match(r'\s*Rectangle\(2\)', line):
            # Calculate new Y = -gap / 2
            y = -new_gap / 2
            new_line = f'Rectangle(2) = {{-50, {y}-2, 0, 100, 4, 0}};\n'
            new_lines.append(new_line)
        else:
            new_lines.append(line)

    # Define the directory and file name
    directory = "meshes/translation"
    file_name = "geometry_distance_" + str(new_gap) + ".geo"
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    # Create the file in the directory
    file_path = os.path.join(directory, file_name)
    with open(file_path, "w") as f:
        f.writelines(new_lines)

# Function to modify overetch of plates

In [3]:
import re
import os

def modify_plates_overetch(overetch):
    with open("geometry.geo", "r") as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        # Match Rectangle A (upper)
        if re.match(r'\s*Rectangle\(1\)', line):
            y = 2*overetch
            thickness = 4-y
            new_line = f'Rectangle(1) = {{-50+{overetch}, 1+{thickness/2}, 0, 100-{y}, {thickness}, 0}};\n'
            new_lines.append(new_line)
        # Match Rectangle B (lower)
        elif re.match(r'\s*Rectangle\(2\)', line):
            new_line = f'Rectangle(2) = {{-50+{overetch}, -1-{thickness/2}, 0, 100-{y}, {thickness}, 0}};\n'
            new_lines.append(new_line)
        else:
            new_lines.append(line)

    # Define the directory and file name
    directory = "meshes/deformation"
    file_name = "geometry_overetch_" + str(overetch) + ".geo"
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    # Create the file in the directory
    file_path = os.path.join(directory, file_name)
    with open(file_path, "w") as f:
        f.writelines(new_lines)

## Function to rotate a rectangle

In [4]:
import re
import os
from math import pi

def rotate_upper_plate(angle):
    with open("geometry.geo", "r") as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        if "Rotate" in line:
            # Replace the last value before the closing } {
            new_line = re.sub(r'(-?\d+(\.\d+)?)\s*\}\s*\{', f'{angle*pi/180} }} {{', line)
            new_lines.append(new_line)
        else:
            new_lines.append(line)

    # Define the directory and file name
    directory = "meshes/rotation"
    file_name = "geometry_angle_" + str(angle) + ".geo"
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    # Create the file in the directory
    file_path = os.path.join(directory, file_name)
    with open(file_path, "w") as f:
        f.writelines(new_lines)

## Generation of different geometries

In [5]:
import numpy as np
def generate_meshes():
    # Generate 40 equispaced values
    distances = np.linspace(1.5, 2.5, 40)
    overetches = np.linspace(0.1, 0.6, 40)
    angles = np.linspace(1, -1, 40)

    for d in distances:
        modify_plates_distance(d)

    for o in overetches:
        modify_plates_overetch(o)

    for a in angles:
        rotate_upper_plate(a)

generate_meshes()
print("Mesh files generated successfully.")

Mesh files generated successfully.


## Mesh generation

In [26]:
import gmsh
import os

# Initialize Gmsh
gmsh.initialize()

# Optional: Hide Gmsh messages
gmsh.option.setNumber("General.Terminal", 1)

# Path to your .geo file
geo_file = "geometry.geo"

# Load the .geo file
gmsh.open(geo_file)

# Generate 2D or 3D mesh depending on your .geo setup
# You can also use gmsh.model.mesh.generate(dim) with dim=1,2,3
gmsh.model.mesh.generate(2)

# Write the mesh to a .msh file
msh_file = os.path.splitext(geo_file)[0] + ".msh"
gmsh.write(msh_file)

# Finalize Gmsh
gmsh.finalize()

print(f"Mesh saved to {msh_file}")

Info    : Reading 'geometry.geo'...


Error   : OpenCASCADE wire or curve loop with tag 4 already exists


Exception: OpenCASCADE wire or curve loop with tag 4 already exists